In [48]:
!pip install --upgrade sqlalchemy
!pip install ipython-sql
!pip install ipython-sql prettytable

In [49]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [50]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [51]:
!pip install -q pandas

In [52]:
%sql sqlite:///my_data1.db

In [53]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [54]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [55]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

Task 1
Display the names of the unique launch sites in the space mission

In [56]:
%sql SELECT DISTINCT LAUNCH_SITE as "Launch_Sites" FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Sites
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


Task 2
Display 5 records where launch sites begin with the string 'CCA'

In [57]:
%sql SELECT * FROM SPACEXTBL WHERE LAUNCH_SITE LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


Task 3
Display the total payload mass carried by boosters launched by NASA (CRS)

In [58]:
import sqlite3
import pandas as pd

# Conectar a la base de datos (se creará si no existe)
con = sqlite3.connect("my_datal.db")
cur = con.cursor()

# Crear la tabla SPACEXTBL si no existe
cur.execute('''
CREATE TABLE IF NOT EXISTS SPACEXTBL (
    Date TEXT,
    Time TEXT,
    Booster_Version TEXT,
    Launch_Site TEXT,
    Payload TEXT,
    PAYLOAD_MASS__KG_ REAL,
    Orbit TEXT,
    Customer TEXT,
    Mission_Outcome TEXT,
    Landing_Outcome TEXT
);
''')

# Cargar datos desde un archivo CSV (si tienes uno)
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Insertar los datos en la tabla SPACEXTBL
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False)

# Guardar los cambios
con.commit()

# Cerrar la conexión
con.close()

In [59]:
import sqlite3

# Conectar a la base de datos
con = sqlite3.connect("my_datal.db")
cur = con.cursor()

# Consulta para calcular la masa total de carga útil de NASA (CRS)
query = '''
SELECT SUM("PAYLOAD_MASS__KG_") AS "Total payload mass by NASA (CRS)" 
FROM "SPACEXTBL" 
WHERE "CUSTOMER" = 'NASA (CRS)';
'''

# Ejecutar la consulta
cur.execute(query)

# Obtener y mostrar el resultado
result = cur.fetchone()
print("Total payload mass by NASA (CRS):", result[0])

# Cerrar la conexión
con.close()

Total payload mass by NASA (CRS): 45596


Task 4
Display average payload mass carried by booster version F9 v1.1

In [68]:
import sqlite3

# Conectar a la base de datos
con = sqlite3.connect("my_datal.db")
cur = con.cursor()

# Consulta 1: Calcular la masa promedio de carga útil para la versión F9 v1.1
query1 = '''
SELECT AVG("PAYLOAD_MASS__KG_") AS "Average payload" 
FROM "SPACEXTBL" 
WHERE "BOOSTER_VERSION" = 'F9 v1.1';
'''

# Ejecutar la consulta 1
cur.execute(query1)
result1 = cur.fetchone()
print("Average payload for F9 v1.1:", result1[0])


# Cerrar la conexión
con.close()

Average payload for F9 v1.1: 2928.4


Task 5
List the date when the first succesful landing outcome in ground pad was acheived.

In [69]:
import sqlite3

# Conectar a la base de datos
con = sqlite3.connect("my_datal.db")
cur = con.cursor()


# Consulta 2: Obtener la fecha del primer aterrizaje exitoso en plataforma terrestre
query2 = '''
SELECT MIN(DATE) AS "First successful landing outcome in ground pad" 
FROM "SPACEXTBL" 
WHERE "LANDING_OUTCOME" = 'Success (ground pad)';
'''

# Ejecutar la consulta 2
cur.execute(query2)
result2 = cur.fetchone()
print("First successful landing outcome in ground pad:", result2[0])

# Cerrar la conexión
con.close()

First successful landing outcome in ground pad: 2015-12-22


Task 6
List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [63]:
%sql SELECT BOOSTER_VERSION FROM SPACEXTBL WHERE LANDING_OUTCOME = 'Success (drone ship)' AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


Task 7
List the total number of successful and failure mission outcomes

In [64]:
%sql SELECT number_of_success_outcomes, number_of_failure_outcomes FROM (SELECT COUNT(*) AS number_of_success_outcomes FROM SPACEXTBL WHERE MISSION_OUTCOME LIKE 'Success%'), (SELECT COUNT(*) number_of_failure_outcomes FROM SPACEXTBL WHERE MISSION_OUTCOME LIKE 'Failure%')

 * sqlite:///my_data1.db
Done.


number_of_success_outcomes,number_of_failure_outcomes
100,1


Task 8
List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [65]:
%sql SELECT DISTINCT BOOSTER_VERSION FROM SPACEXTBL WHERE PAYLOAD_MASS__KG_ =(SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


Task 9
List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [66]:
%%sql SELECT 
    substr(DATE, 6, 2) AS month,
    BOOSTER_VERSION, 
    LAUNCH_SITE 
FROM SPACEXTBL 
WHERE substr(DATE, 1, 4) = '2015' 
AND LANDING_OUTCOME = 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


month,Booster_Version,Launch_Site
01,F9 v1.1 B1012,CCAFS LC-40
04,F9 v1.1 B1015,CCAFS LC-40


Task 10
Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [67]:
%sql SELECT LANDING_OUTCOME, COUNT(LANDING_OUTCOME) AS Landing_Count FROM SPACEXTBL WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY  LANDING_OUTCOME ORDER BY COUNT(LANDING_OUTCOME) DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Landing_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
